In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import eegraph
import stellargraph as sg
from stellargraph import StellarGraph
import os
import pickle


## Variables a usar

In [2]:
path = '../GCNN/Scripts/data/data_vigilia'              #<--------------- PATH TO FOLDER CONTAINING EEGs
window_size_class_0 = 1                                 #<--------------- CLASS 0 WINDOW SIZE
window_size_class_1 = 1                               #<--------------- CLASS 1 WINDOW SIZE
connectivity_number_total = 1                                
exclude = [] 
#CONNECTIVITY MEASURES USED
connectivity= "squared_coherence"
bands = ['theta', 'alpha', 'beta']

## Funcion para convertir eegraf a StellarGraph

In [3]:
def convertir_eegraph(G):
    array_grafos= []
    #recorremos los grafos
    for i in range(len(G)):
        #convertimos el grafo
        s = StellarGraph.from_networkx(G[i])
        #los guardamos en un array
        array_grafos.append(s)
    return array_grafos


## funcion modelar Grafo

In [4]:
def modelate_with_different_connectivity(window_size, G, connectivity_number_total, connectivity, bands):
    array_grafos = []
    #recorremos el numero de conectividades elegidas
    for i in range(connectivity_number_total):
        #creamos nuestro grafo modelado y guardamos nuestra matriz de adyacencia(no nos hace falta pero son datos que podrían venir bien en un futuro)
        graphs, c_m = G.modelate(window_size = window_size, connectivity = connectivity, bands = bands) #<--------------- THRESHOLD , threshold = 0.2          
        #llamamos recursivamente a la función para convertir los grafos de tipo networkx en un objeto manejable por Stellar graph
        array_grafos = array_grafos + convertir_eegraph(graphs)
    return  array_grafos

## Abrir directorios e importar datos

In [5]:
def open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands):
    array_grafos = []
    class_files = os.listdir(path)
    #recorremos todas las carpetas y ficheros del path dado
    for entry in class_files:
        eeg_files = os.listdir(path + '/' + entry)
        for eeg in eeg_files:
            eeg_path = (path + '/' + entry + '/' + eeg)
            print(eeg_path, entry)#imprimimos el path del archivo completo
            G = eegraph.Graph() #creamos un objeto de tipo eegraph
            G.load_data(path= eeg_path, exclude = exclude) # llamamos al objeto y cargamos el archivo seleccionado
            
            if(entry == '1'):
                window_size = window_size_class_1
            elif (entry == '0'):
                window_size = window_size_class_0
            #llamamos recursivamente a la función para modelar nuestro grafo con las medidas deseadas
            array_grafos = array_grafos + modelate_with_different_connectivity(window_size , G, connectivity_number_total,connectivity,bands)
        
    return array_grafos

## llamada a las funciones y guardado de datos

In [6]:
graphs = open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands)


with open("../Datos/graphs.txt", "wb") as fp:   # Pickling
    pickle.dump(graphs, fp)


../GCNN/Scripts/data/data_vigilia/0/1_vigilia con 1.edf 0
Extracting EDF parameters from d:\CEIEC\EEG-Epilepsy\EEG-Epilepsy\GCNN\Scripts\data\data_vigilia\0\1_vigilia con 1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 25
Sample rate: 512.0 Hz.
Duration: 4.998 seconds.
Channel Names: ['EEG Fp1', 'EEG Fp2', 'EEG F3', 'EEG F4', 'EEG C3', 'EEG C4', 'EEG P3', 'EEG P4', 'EEG O1', 'EEG O2', 'EEG F7', 'EEG F8', 'EEG T3', 'EEG T4', 'EEG T5', 'EEG T6', 'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG FT9', 'EEG FT10', 'EEG T9', 'EEG T10', 'EEG TP9', 'EEG TP10']
Model Data.
Squared_coherence_Estimator()
Frequency Bands: ['delta', 'theta', 'alpha', 'beta', 'gamma'] [False, True, True, True, False]
Intervals:  [(0, 512.0), (512.0, 1024.0), (1024.0, 1536.0), (1536.0, 2048.0), (2048.0, 2559.0)]

Threshold: 0.65
Number of graphs created: 15
../GCNN/Scripts/data/data_vigilia/0/1_vigilia con 10.edf 0
Extracting EDF parameters from d:\C